In [39]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from modelAlpha import _netlocalWD,_netWG
import utils

In [69]:
class opt():
    def __init__(self):
        self.dataset= 'streetview'
        self.testimg = 'test_toon8.png'
        self.workers=2
        self.batchSize=64 #'input batch size')
        self.imageSize=256 #the height / width of the input image to network')
        self.nz=100
        self.ngf=128
        self.ndf=128 # center image size
        self.nc=4
        self.niter=1
        self.lr=0.0002
        self.beta1=0.5
        self.cuda=True
        self.ngpu=1
        self.netG='model/netG_streetview_1.pth'
        #self.netG='model/tmp.pth'
        self.outf='.'
        self.manualSeed = 0
        self.nBottleneck=4000  # 'of dim for bottleneck of encoder')
        self.overlapPred = 0 # 'overlapping edges')
        self.nef = 64#'of encoder filters in first conv layer')
        self.wtl2 = 0.998 #'0 means do not use else use with this weight')
        self.wtlD =0.001# means do not use else use with this weight')
        self.jittering=False


In [70]:
opt = opt()

In [42]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


netG = _netWG(opt)
netG.apply(weights_init)
netG.load_state_dict(torch.load(opt.netG,map_location=lambda storage, location: storage)['state_dict'])
resume_epoch = torch.load(opt.netG)['epoch']
netG.eval()
print(netG)

_netWG(
  (main): Sequential(
    (0): Conv2d(4, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(0.2, inplace)
    (2): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU(0.2, inplace)
    (5): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU(0.2, inplace)
    (8): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU(0.2, inplace)
    (11): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (12): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (13): LeakyReLU(0.2, inplace)
    (14): Conv2d(512, 4000, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (15): BatchNorm2d(4000, eps=

In [71]:
image_margin = int((opt.imageSize - opt.ndf)/2)
transform = transforms.Compose([transforms.ToTensor()])

png_image = utils.load_image(opt.testimg, opt.imageSize)
png_image = transform(png_image)
input_cropped = torch.FloatTensor(1, 4, opt.imageSize, opt.imageSize)
input_pngReverse = torch.FloatTensor(4, opt.ndf, opt.ndf)
result_img = torch.FloatTensor(1, 3, opt.imageSize, opt.imageSize)
input_cropped = Variable(input_cropped)



#input_cropped.data.resize_(png_image.size()).copy_(png_image)
input_pngReverse[0] = png_image[3][image_margin:image_margin+opt.ndf, image_margin:image_margin+opt.ndf]
png_imageV = Variable(png_image)
input_cropped.data[0,0] = png_imageV.data[0]
input_cropped.data[0,1] = png_imageV.data[1]
input_cropped.data[0,2] = png_imageV.data[2]
vutils.save_image(input_pngReverse[0],'result/single/cropped.png')
input_cropped.data[0,0,
                  image_margin:image_margin+opt.ndf,
                  image_margin:image_margin+opt.ndf] = input_pngReverse[0] * input_cropped.data[0,0,
                  image_margin:image_margin+opt.ndf,
                  image_margin:image_margin+opt.ndf]
input_cropped.data[0,1,
                  image_margin:image_margin+opt.ndf,
                  image_margin:image_margin+opt.ndf] = input_pngReverse[0] * input_cropped.data[0,1,
                  image_margin:image_margin+opt.ndf,
                  image_margin:image_margin+opt.ndf]
input_cropped.data[0,2,
                  image_margin:image_margin+opt.ndf,
                  image_margin:image_margin+opt.ndf] = input_pngReverse[0] * input_cropped.data[0,2,
                  image_margin:image_margin+opt.ndf,
                  image_margin:image_margin+opt.ndf]
input_cropped.data[0,3] = png_image[3]
vutils.save_image(input_cropped.data[0,0],'result/single/tmp1.png')
vutils.save_image(input_cropped.data[0,1],'result/single/tmp2.png')
vutils.save_image(input_cropped.data[0,2],'result/single/tmp3.png')
vutils.save_image(input_cropped.data[0,3],'result/single/tmp4.png')
input_pngReverse[0] = torch.abs(png_image[3] - 1)[image_margin:image_margin+opt.ndf, image_margin:image_margin+opt.ndf]
print("data loading has been done")

data loading has been done


In [72]:
epoch = resume_epoch
vutils.save_image(input_cropped.data[0],'tmp.png')

# train with fake
fake = netG(input_cropped)

result_img[0,0] = input_cropped.data[0,0]
result_img[0,1] = input_cropped.data[0,1]
result_img[0,2] = input_cropped.data[0,2]
vutils.save_image(result_img[0],'result/single/single_test_image(cropped)_%03d_%s' % (epoch,opt.testimg))
result_img[0,0,
     image_margin:image_margin+opt.ndf,
     image_margin:image_margin+opt.ndf] += fake.data[0,0] * input_pngReverse[0]
result_img[0,1,
     image_margin:image_margin+opt.ndf,
     image_margin:image_margin+opt.ndf] += fake.data[0,1] * input_pngReverse[0]
result_img[0,2,
     image_margin:image_margin+opt.ndf,
     image_margin:image_margin+opt.ndf] += fake.data[0,2] * input_pngReverse[0]
vutils.save_image(result_img[0],'result/single/single_test_image(mask)_%03d_%s' % (epoch,opt.testimg))
result_img[0,0] = input_cropped.data[0,0]
result_img[0,1] = input_cropped.data[0,1]
result_img[0,2] = input_cropped.data[0,2]
result_img[0,0,
     image_margin:image_margin+opt.ndf,
     image_margin:image_margin+opt.ndf] = fake.data[0,0]
result_img[0,1,
     image_margin:image_margin+opt.ndf,
     image_margin:image_margin+opt.ndf] = fake.data[0,1]
result_img[0,2,
     image_margin:image_margin+opt.ndf,
     image_margin:image_margin+opt.ndf] = fake.data[0,2]

vutils.save_image(result_img[0],'result/single/single_test_image(recon)_%03d_%s' % (epoch,opt.testimg))
